In [1]:
#import libraries
import numpy as np
import pprint
import scipy
import scipy.linalg
import pandas as pd
import random
from collections import OrderedDict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
import matplotlib.pyplot as pl
import json
from tqdm import tqdm
from collections import defaultdict
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise.model_selection import train_test_split
np.set_printoptions(threshold=20)
pd.options.mode.copy_on_write = True

In [2]:
#FUNCTION TO CONVERT CSV TO DATAFRAME
#converts and reduces dataframe to usable data
def csv_to_df(string1, string2):    
    df = pd.read_csv(string1)

    df_book = pd.read_csv(string2)

    category_array = df_book['categories'].unique()

    category_array.size
    #print('\n'.join(category_array.values))



    #Preprocessing/Data Cleaning
    df_ratings = df

    drop_columns = ["Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

    books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

    #Create Dictionary Key for Unique ID:Book
    book_dict = dict()
    for row in books_ratings_cleaned.itertuples():
        if row[1] not in book_dict:
            book_dict[row[1]] = [row[2],0]

        book_dict[row[1]][1] +=1


    drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

    books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

    #books_ratings_cleaned

    books_ratings_cleaned['User_id'].isna()

    sum(books_ratings_cleaned['User_id'].isna())

    books_ratings_cleaned = books_ratings_cleaned[books_ratings_cleaned['User_id'].notna()]

    return books_ratings_cleaned, book_dict

In [3]:
#function to convert data frame to numpy matrix
def Convert_to_Matrix(matrix):
    #convert Dictionary Key into Sparse Matrix With users as Columns, Books as rows
    test = pd.DataFrame.from_dict(matrix)

    test.head()

    #take Matrix Transpose
    test_transpose = test.transpose()

    #populate NaN's as zeroes
    A_transpose = test_transpose.fillna(0)

    return A_transpose

#function to perform LU factorization on numpy matrix
def LU_Factorization(matrix):
    #A = PLU FACTORIZATION
    P, L, U = scipy.linalg.lu(matrix)

    A_test = np.matmul(P, np.matmul(L, U))

    #Test for Accuracy
    print(np.allclose(matrix, A_test))
    
    return P, L, U
    

In [4]:
#COLLABORATIVE FILTERING COSINE SIMILARITY FUNCTION
#finding similar users by using cosine similarity algorithm
def find_similar(user_1, k):
    allusers = A.values
    denominator1 = np.sqrt(sum([np.square(x) for x in user_1]))
    
    #performs cosine similarity algorithm on vectors (users)
    cosinesimilarity = [(user_1.name,1)]
    i=1
    for user in tqdm(allusers[1:]):
        numerator = [x*y for x,y in zip(user_1.values, user)]
        denominator2 = np.sqrt(sum([np.square(x) for x in user]))
        costheta = sum(numerator) / (denominator1 * denominator2)
        cosinesimilarity.append((A.index[i],costheta))
        i+=1
    
    #sort the results
    cosinesimilarity.sort(key = lambda x: x[1], reverse = True)
    
    #combine sorted results
    similar10users = cosinesimilarity[0:k]
    
    #output results of sorted similar users
    for i in range(0,k):
        print(similar10users[i])
    
    #place similar users into a data frame
    top10usersdf = pd.DataFrame()
    for user in similar10users:
        top10usersdf = top10usersdf.append(A.loc[user[0]])
    top10usersdf['costheta'] = [user[1] for user in similar10users]
    
    #to be used in calculation of distance/similarity for recommendation
    all_values = top10usersdf.values

    return top10usersdf

In [27]:
# Some helper methods for getting subsets of users that have more data, reducing the sparcity of matrix
# when experimenting with the dataset
def get_most_frequent_users(count, source_frame):
    assert isinstance(count, int), "Argument must be an integer"
    assert count > 0, "Argument must be a positive integer"
    
    user_occurences = source_frame['User_id'].value_counts()
    
    assert count < user_occurences.count(), "Requested more data then we have...not tiny"
    
    return user_occurences.head(count).index.to_list()

def get_users_with_minimal_ratings(rated, source_frame):
    assert isinstance(rated, int), "Argument must be an integer"
    assert rated > 0, "Argument must be a positive integer"
    
    # Get a series mapping user ids to their occurences in the data frame
    user_occurences = source_frame['User_id'].value_counts()
    return user_occurences[user_occurences > rated].index.to_list()

def get_training_data(ratio: float, source_frame):
    assert isinstance(ratio, float), "Ratio must be a float"
    assert 0.0 < ratio < 1.0, "Ratio must be in [0, 1]"
    
    # we want to ensure that our test data is built from good users, users with
    # less then 10 ratings are considered un-testable.
    # FILTER USERS BY NUMBER OF REVIEWS PER USER HERE
    good_users = get_users_with_minimal_ratings(20, source_frame)
    
    # Randomize the order of ratings for users in our good users set.
    good_users_ratings = source_frame[source_frame['User_id'].isin(good_users)].sample(frac=1)
    
    total_count = good_users_ratings.shape[0]
    training_data_count = int(total_count * ratio)
    
    training_data = pivot_rating_to_user_frame(good_users_ratings[0:training_data_count])
    test_data = pivot_rating_to_user_frame(good_users_ratings[training_data_count:])
    
    return training_data, test_data

def pivot_rating_to_user_frame(source_frame):
    new_data = dict()
    for _, row in tqdm(source_frame.iterrows(), total=source_frame.shape[0]):
        (user_id, book_id, score) = row
        if user_id not in new_data:
            new_data[user_id] = defaultdict(lambda: np.nan)
        new_data[user_id][book_id] = score
        
    return pd.DataFrame.from_dict(new_data)

In [28]:
books_ratings_cleaned, book_dict = csv_to_df("Books_rating.csv", "books_data.csv")

In [29]:
books_ratings_cleaned.count()

Id              2438213
User_id         2438213
review/score    2438213
dtype: int64

In [30]:
columns_titles = ["User_id","Id", "review/score"]
Books_ratings_cleaned = books_ratings_cleaned.reindex(columns=columns_titles)

Books_ratings_cleaned.head()

,User_id,Id,review/score
0,AVCGYZL8FQQTD,1882931173,4.0
1,A30TK6U7DNS82R,0826414346,5.0
2,A3UH4UZ4RSVO82,0826414346,5.0
3,A2MVUWT453QH61,0826414346,4.0
4,A22X4XUPKF66MR,0826414346,4.0


In [31]:
#EXCUTE ME PLEASE FOR REAL IM DYING
#LIKE REALLY, RUN ME
training_data, test_data = get_training_data(0.75, books_ratings_cleaned.sample(100000))

100%|██████████████████████████████████████████████████████████████████████████████| 559/559 [00:00<00:00, 9643.15it/s]


In [32]:
A = training_data.fillna(0)

A.shape

#A = A.to_numpy()

#np.linalg.eig(A)

(60, 1546)

In [33]:
# read in values as Surprise dataset
#reader = Reader(rating_scale=(1, 5))
#data = Dataset.load_from_df(training_data, reader)

In [34]:
#A_similar = find_similar(A.iloc[0],10)

#A_similar

In [35]:
#USE THIS TO CHECK BOOK ID
#print(book_dict.get('1557424470'))
#A_similar.head()

In [60]:
USERBOOK = Books_ratings_cleaned.sample(10000)

#USERBOOK.head()

In [61]:
reader = Reader()
data = Dataset.load_from_df(USERBOOK, reader)

In [62]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1678  1.1254  1.1315  1.1500  1.1671  1.1483  0.0175  
MAE (testset)     0.9090  0.8811  0.8835  0.8970  0.9045  0.8950  0.0111  
Fit time          0.17    0.19    0.25    0.18    0.20    0.20    0.03    
Test time         0.03    0.02    0.03    0.02    0.02    0.02    0.01    


{'test_rmse': array([1.16775746, 1.12542127, 1.13152391, 1.14996302, 1.16706174]),
 'test_mae': array([0.90903517, 0.88108647, 0.88349131, 0.89700334, 0.90453887]),
 'fit_time': (0.17227768898010254,
  0.18749475479125977,
  0.24895572662353516,
  0.18121576309204102,
  0.2031714916229248),
 'test_time': (0.031241416931152344,
  0.015412569046020508,
  0.02881455421447754,
  0.015622377395629883,
  0.017634868621826172)}

In [63]:
dataset = data.build_full_trainset()
svd = SVD(n_factors=5000, reg_all=0.05)
svd.fit(dataset)

In [64]:
print(svd.predict('A30TK6U7DNS82R', '0826414346'))
print(book_dict.get('0826414346'))

user: A30TK6U7DNS82R item: 0826414346 r_ui = None   est = 4.26   {'was_impossible': False}
['Dr. Seuss: American Icon', 9]


In [65]:
svd.predict('A3UH4UZ4RSVO82', '0826414346')

Prediction(uid='A3UH4UZ4RSVO82', iid='0826414346', r_ui=None, est=4.2497, details={'was_impossible': False})

In [66]:
svd.predict('A2MVUWT453QH61', '0826414346')

Prediction(uid='A2MVUWT453QH61', iid='0826414346', r_ui=None, est=4.2497, details={'was_impossible': False})

In [67]:
#sorted_dict = sorted(book_dict.items(), key=lambda e:e[1])

In [68]:
svd.predict('AVCGYZL8FQQTD', '1882931173')

Prediction(uid='AVCGYZL8FQQTD', iid='1882931173', r_ui=None, est=4.2497, details={'was_impossible': False})

In [69]:
trainset, testset = train_test_split(data, test_size=0.20)
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.1725


1.1725398137881198

In [70]:
book_dict

{'1882931173': ['Its Only Art If Its Well Hung!', 1],
 '0826414346': ['Dr. Seuss: American Icon', 9],
 '0829814000': ['Wonderful Worship in Smaller Churches', 4],
 '0595344550': ['Whispers of the Wicked Saints', 32],
 '0253338352': ['Nation Dance: Religion, Identity and Cultural Difference in the Caribbean',
  1],
 '0802841899': ['The Church of Christ: A Biblical Ecclesiology for Today', 4],
 'B0007FIF28': ['The Overbury affair (Avon)', 1],
 'B000JINSBG': ['A Walk in the Woods: a Play in Two Acts', 3],
 '0895554224': ['Saint Hyacinth of Poland', 2],
 '0963923080': ["Rising Sons and Daughters: Life Among Japan's New Young", 3],
 '0854968350': ["Muslim Women's Choices: Religious Belief and Social Reality (Cross Cultural Perspectives on Women)",
  2],
 '0918973031': ['Dramatica for Screenwriters', 10],
 '1858683092': ['Mensa Number Puzzles (Mensa Word Games for Kids)', 2],
 '0792391810': ['Vector Quantization and Signal Compression (The Springer International Series in Engineering and Com